# Importing Packages

In [ ]:
import os
os.chdir('/content/LG_ML_tutorial/2. Finding optimal settings')

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler

# Diamond dataset

Data from https://www.adiamor.com/Diamonds/Search

2,690 observations

+ predictors
   + carat.size: numeric
   + color: ordinal, 0 ~ 7 (colorless to nearly colorless)
   + clarity: ordinal, 0 ~ 6 (internally flawless to slightly included)
   + cut: ordinal, 0 ~ 3 (ideal to good)   
+ response
   + price: diamond price

## The optimization problem we have to solve in real life is like below.
You would like to purchase a diamond.

Your budget is 400 USD.

She wants that the color of diamond should be at level of 1.

She wants that the clarity of diamond should be at level of 2.

**(Question) What size and which cut level of diamond can you purchase?**


In [42]:
diamond = pd.read_csv('diamond.csv')

min_max_scaler = MinMaxScaler()
diamond_norm = min_max_scaler.fit_transform(diamond)

mins = min_max_scaler.data_min_
maxs = min_max_scaler.data_max_

In [48]:
# -------------------------------------------------------------------------- #
# response estimation using neural network
# -------------------------------------------------------------------------- #
model = MLPRegressor(solver='adam', alpha=1e-5, hidden_layer_sizes=(6, 6))

X = diamond_norm[:,:-1]
Y = diamond_norm[:,-1]

model.fit(X,Y)

MLPRegressor(alpha=1e-05, hidden_layer_sizes=(6, 6))

In [49]:
price_hat = model.predict(X)
orig_price_hat = price_hat * (maxs[4]-mins[4]) + mins[4]
orig_price = np.array(diamond['price'])
R2 = 1-np.sum((orig_price-orig_price_hat)**2)/np.sum((orig_price-np.mean(orig_price_hat))**2) # R-squared

In [50]:
# -------------------------------------------------------------------------- #
# optimization problem example
#       1. You want to purchase a diamond.
#       2. Your budget is 400 USD.
#       3. The color of your diamond should be at level of 1.
#       4. The clarity of your diamond should be at level of 2.
#       Q. What size and which cut level of diamond can you purchase?
# -------------------------------------------------------------------------- #

target_norm = (400-mins[4])/(maxs[4]-mins[4])  # 400 USD
color_norm = (1-mins[1])/(maxs[1]-mins[1])  # color level is 1 (given)
clarity_norm = (2-mins[2])/(maxs[2]-mins[2]) # clarity level is 2 (given)

In [52]:
sample_row = diamond_norm[0,:-1]
sample_row[1] = color_norm
sample_row[2] = clarity_norm

In [54]:
# -------------------------------------------------------------------------------------- #
# loss function
#   input
#              model: prediction model
#       x.values.all: predictor variable values, some of those are given,
#                     those corresponding to idx will change during optimization procedure
#             target: we want to get x.opt values such that the response reaches at this value
#              x.opt: predictor variable values to be optimized
#                idx: indices corresponding to x.opt variables
#
#   output
#               loss: loss function value (we want to minimize this)
#
# CREATED 12/16/2018
# -------------------------------------------------------------------------------------- #

def loss_fun(x_opt, model, x_values_all, target, idx ):
  x_values_all[:,idx] = x_opt
  pred = model.predict( x_values_all )
  loss = (pred - target)**2 # squared loss function
  return(loss)

In [55]:
from scipy import optimize

x0 = np.random.uniform(0,1,2).reshape(1, -1)
sample_row = sample_row.reshape(1, -1)
bounds = optimize.Bounds([0,0],[1,1])
res = optimize.minimize(loss_fun,x0,args=(model,sample_row,target_norm,np.array([0,3])),method='L-BFGS-B', bounds=bounds,options={'disp': True})

In [58]:
sample_row[:,0] = res.x[0]
sample_row[:,3] = res.x[1]
target_hat = model.predict(sample_row)
orig_target_hat = target_hat* (maxs[4]-mins[4]) + mins[4]
print(orig_target_hat)
# ** we can see that the price is 400 at the optimal solution, which means that we correctly found the optimal solution


# answer to the question Q
opt_carat_size = res.x[0] * (maxs[0]-mins[0]) + mins[0] # carat size in original scale
opt_cut = res.x[1] * (maxs[3]-mins[3]) + mins[3] # cut in original scale

print([opt_carat_size,opt_cut])

[399.99999141]
[0.6807781857498151, 0.17297610489451848]
